In [36]:
## VISUALIZATION OF ALL DISTRIBUTION

In [1]:
%load_ext autoreload
%autoreload 2


import pandas as pd
import synapseclient as sc
import sys
sys.path.append("../../src")
from utils.munging_utils import get_file_entity, fix_column_name, save_data_to_synapse
from utils.preprocessing_utils import preprocess, addAdditionalFeatures_viz, collapseFeatures
from datetime import datetime
import numpy as np

Welcome, aryton tediarjo!



In [2]:
MPOWER_GAIT_DATA_V1 = "syn21111818"
MPOWER_DEMO_DATA_V1 = "syn10371840"
MPOWER_GAIT_DATA_V2 = "syn21113231"
MPOWER_DEMO_DATA_V2 = "syn15673379"
MPOWER_GAIT_DATA_PASSIVE = "syn21114136"
EMS_PROF_DATA = "syn10235463"
EMS_DEMO_DATA = "syn10295288"
EMS_GAIT_DATA = "syn21256442"
EMS_PASSIVE_DATA = "syn10651116"

In [3]:
syn = sc.login()

Welcome, aryton tediarjo!



In [5]:
test = get_file_entity(MPOWER_GAIT_DATA_V1)

In [ ]:
metadata_cols = ['recordId', 'healthCode', 'appVersion', 
                 'phoneInfo', 'createdOn', 'PD', 'MS',
                 'gender', 'age', 'version']
data = data[[feat for feat in data.columns if ("." in feat) or (feat in metadata_cols)]]

In [7]:
## gait mPower V1 ##

def create_mPowerV1_data(MPOWER_DATA_V1, MPOWER_DEMO_V1, output_filename):
    demo_data = syn.tableQuery("SELECT * FROM {} where dataGroups\
                               NOT LIKE '%test_user%'".format(MPOWER_DEMO_V1)).asDataFrame()
    gait_data = get_file_entity(MPOWER_DATA_V1)
    demo_data = demo_data[["healthCode", "gender", "age",
                           "professional_diagnosis", "inferred_diagnosis"]].reset_index(drop = True)
    data = pd.merge(gait_data, demo_data, on = "healthCode", how = "inner")
    data_return   = data[[feature for feature in data.columns if "outbound" not in feature]]
    data_outbound = data[[feature for feature in data.columns if "return" not in feature]]
    data = pd.concat([fix_column_name(data_outbound), fix_column_name(data_return)])
    data = data.dropna(subset = ["inferred_diagnosis"], thresh = 1)
    data["PD"] = data["inferred_diagnosis"].map({True :1.0, False:0.0})
    data = data[(data["gender"] == "Female") | (data["gender"] == "Male")]
    data["age"] = data["age"].apply(lambda x: float(x))
    data = data[(data["age"] <= 100) & (data["age"] >= 0)]
    data["gender"] = data["gender"].apply(lambda x: x.lower())
    data = fix_column_name(data)
    data = data.reset_index(drop = True)
    metadata_cols = ['recordId', 'healthCode', 'appVersion', 
                 'phoneInfo', 'createdOn', 'PD', 'MS',
                 'gender', 'age', 'version']
    data = data[[feat for feat in data.columns if ("." in feat) or (feat in metadata_cols)]]
    save_data_to_synapse(data = data, 
                         output_filename = "mpower_v1_active_full.csv",
                         data_parent_id = "syn21267355")
    return data 

In [8]:
def create_mPowerV2_data(MPOWER_DATA_V2, MPOWER_DEMO_V2, output_filename):

    demo_data = syn.tableQuery("SELECT birthYear, healthCode, diagnosis, sex FROM {} \
                                where dataGroups NOT LIKE '%test_user%'".format(MPOWER_DEMO_V2)).asDataFrame()
    gait_data = get_file_entity(MPOWER_DATA_V2)
    data   = pd.merge(gait_data, demo_data, how = "inner", on = "healthCode")
    data   = data[data["diagnosis"] != "no_answer"] 
    data["PD"] = data["diagnosis"].map({"parkinsons":1, "control":0})
    data["age"] = data["birthYear"].apply(lambda year: datetime.now().year - year)
    data = data.rename({"sex":"gender"}, axis = 1)
    data = fix_column_name(data)
    data = data.reset_index(drop = True)
    metadata_cols = ['recordId', 'healthCode', 'appVersion', 
                 'phoneInfo', 'createdOn', 'PD', 'MS',
                 'gender', 'age', 'version']
    data = data[[feat for feat in data.columns if ("." in feat) or (feat in metadata_cols)]]
    save_data_to_synapse(data = data, output_filename = output_filename,
                              data_parent_id = "syn21267355")
    return data

In [9]:
def create_elevateMS_data(EMS_DATA, EMS_PROF_DATA, output_filename):
    profile_data = syn.tableQuery("SELECT healthCode, dataGroups, 'demographics.gender', 'demographics.age' FROM {}\
                           where dataGroups NOT LIKE '%test_user%'".format(EMS_PROF_DATA)).asDataFrame()
    gait_data    = get_file_entity(EMS_DATA)
    data         = pd.merge(gait_data, profile_data, how = "inner", on = "healthCode")
    data = data.dropna(subset = ["demographics.gender"])
    data["MS"] = data["dataGroups"].map({"ms_patient":1, "control":0})
    data  = data.rename({"demographics.gender" :"gender",
                         "demographics.age"    : "age"}, axis = 1)
    data = fix_column_name(data)
    data = data.reset_index(drop = True)
    metadata_cols = ['recordId', 'healthCode', 'appVersion', 
                 'phoneInfo', 'createdOn', 'PD', 'MS',
                 'gender', 'age', 'version']
    data = data[[feat for feat in data.columns if ("." in feat) or (feat in metadata_cols)]]
    save_data_to_synapse(data = data, output_filename = output_filename,
                                data_parent_id = "syn21267355")
    return data

In [24]:
dataV1                    = create_mPowerV1_data(MPOWER_GAIT_DATA_V1, MPOWER_DEMO_DATA_V1, "pdkit_mpowerv1_active_full.csv")
dataV1["version"]         = "V1"
dataV2                    = create_mPowerV2_data(MPOWER_GAIT_DATA_V2, MPOWER_DEMO_DATA_V2, "pdkit_mpowerv2_active_full.csv")
dataV2["version"]         = "V2"
dataPassive               = create_mPowerV2_data(MPOWER_GAIT_DATA_PASSIVE, MPOWER_DEMO_DATA_V2, "pdkit_mpowerv2_passive_full.csv")
dataPassive["version"]    = "PD_passive"
dataEMS_active            = create_elevateMS_data(EMS_GAIT_DATA, EMS_PROF_DATA, "pdkit_ems_active_full.csv")
dataEMS_active["version"] = "MS"


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################



In [25]:
### some data annotations ### 
data = pd.concat([dataV1, dataV2, dataPassive, dataEMS_active]).reset_index(drop = True)
metadata_cols = ['recordId', 'healthCode', 'appVersion', 
                 'phoneInfo', 'createdOn', 'PD', 'MS',
                 'gender', 'age', 'version']
data = data[[feat for feat in data.columns if ("." in feat) or (feat in metadata_cols)]]
data = data[(data != "#ERROR").all(axis = 1)]
data["is_control"] = data.apply(lambda x: 0 if ((x["PD"] == 0) or (x["MS"] == 0)) else 1, axis = 1)
data[[_ for _ in data.columns if "." in _]] = \
    data[[_ for _ in data.columns if "." in _]].apply(pd.to_numeric)
data.drop(["y.duration", "z.duration", "AA.duration"], axis = 1, inplace = True) 
data.rename({"x.duration": "duration"}, axis = 1, inplace = True)
data = addAdditionalFeatures_viz().transform(data)

save_data_to_synapse(data = data.reset_index(drop = True), output_filename = "combined_gait_data.csv",
                                data_parent_id = "syn21267355")

[WARNING] /Users/atediarjo/Documents/SageBionetworks/synapse_venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  




##################################################
 Uploading file to Synapse storage 
##################################################



In [29]:
for i in syn.getChildren(parent = "syn21267355"):
    if i["name"] == 'combined_gait_data.csv':
        data = utils.get_file_entity(i["id"])
        featurized_healthcode_list = data["healthCode"].unique()

In [33]:
new_data = (utils.get_file_entity(EMS_GAIT_DATA))[~new_data.isin(featurized_healthcode_list)]

In [35]:
comb_data = pd.concat([data, new_data]).reset_index(drop = True)

[WARNING] /Users/atediarjo/Documents/SageBionetworks/synapse_venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.

